# Model Training - Driver Drowsiness Detection

This notebook implements the full training pipeline for the driver drowsiness detection model:
- Data loading and preprocessing
- Train/validation/test splits
- PyTorch CNN model training
- Weights & Biases (W&B) integration for experiment tracking
- Model checkpointing
- Evaluation metrics


In [ ]:
import os
import sys
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import wandb
from dotenv import load_dotenv

# Add src to path
PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / "src"))

# Load environment variables
load_dotenv(PROJECT_ROOT / ".env")

from src.backend.models import DrowsinessCNN, create_model
from src.config.settings import WANDB_PROJECT, WANDB_API_KEY, MODEL_PATH, MODEL_INPUT_SIZE

print(f"Project root: {PROJECT_ROOT}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


In [ ]:
# Initialize W&B
wandb.login(key=WANDB_API_KEY)
wandb.init(
    project=WANDB_PROJECT,
    name="drowsiness_cnn_training",
    config={
        "learning_rate": 0.001,
        "batch_size": 32,
        "epochs": 20,
        "model_architecture": "DrowsinessCNN",
        "input_size": MODEL_INPUT_SIZE,
        "num_classes": 2,
        "optimizer": "Adam",
    }
)


In [ ]:
# Dataset class
class DrowsinessDataset(Dataset):
    def __init__(self, drowsy_dir, non_drowsy_dir, transform=None):
        self.transform = transform
        self.images = []
        self.labels = []
        
        # Load Drowsy images (label 1)
        drowsy_path = Path(drowsy_dir)
        for img_path in drowsy_path.glob("*.png"):
            self.images.append(str(img_path))
            self.labels.append(1)  # drowsy = 1
        
        # Load Non Drowsy images (label 0)
        non_drowsy_path = Path(non_drowsy_dir)
        for img_path in non_drowsy_path.glob("*.png"):
            self.images.append(str(img_path))
            self.labels.append(0)  # alert = 0
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        # Load image
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Data transforms
train_transform = transforms.Compose([
    transforms.Resize(MODEL_INPUT_SIZE),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(MODEL_INPUT_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
# Load dataset
DATA_DIR = PROJECT_ROOT / "Data"
drowsy_dir = DATA_DIR / "Drowsy"
non_drowsy_dir = DATA_DIR / "Non Drowsy"

full_dataset = DrowsinessDataset(drowsy_dir, non_drowsy_dir, transform=None)

# Split dataset: 70% train, 15% val, 15% test
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Apply transforms
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform
test_dataset.dataset.transform = val_transform

# Create data loaders
batch_size = wandb.config.batch_size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print(f"Train samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


In [ ]:
# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model(num_classes=2)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)

print(f"Model created on device: {device}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")


In [ ]:
# Training function
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

# Validation function
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100 * correct / total
    
    # Calculate precision, recall, F1
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average="weighted", zero_division=0
    )
    
    return epoch_loss, epoch_acc, precision, recall, f1


In [ ]:
# Training loop
epochs = wandb.config.epochs
best_val_acc = 0.0
MODELS_DIR = PROJECT_ROOT / "models"
MODELS_DIR.mkdir(exist_ok=True)

for epoch in range(epochs):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc, precision, recall, f1 = validate(model, val_loader, criterion, device)
    
    # Log to W&B
    wandb.log({
        "epoch": epoch,
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc,
        "val_precision": precision,
        "val_recall": recall,
        "val_f1": f1,
    })
    
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"  Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        model_path = MODELS_DIR / "best_model.pth"
        torch.save({
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "epoch": epoch,
            "val_acc": val_acc,
        }, model_path)
        print(f"  ✓ Saved best model (val_acc: {val_acc:.2f}%)")
        
        # Log model artifact to W&B
        artifact = wandb.Artifact("best_model", type="model")
        artifact.add_file(str(model_path))
        wandb.log_artifact(artifact)

print(f"\nTraining completed! Best validation accuracy: {best_val_acc:.2f}%")


In [ ]:
# Load best model and evaluate on test set
model.load_state_dict(torch.load(MODELS_DIR / "best_model.pth")["model_state_dict"])
test_loss, test_acc, test_precision, test_recall, test_f1 = validate(model, test_loader, criterion, device)

print(f"Test Results:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_acc:.2f}%")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall: {test_recall:.4f}")
print(f"  F1 Score: {test_f1:.4f}")

# Log test metrics to W&B
wandb.log({
    "test_loss": test_loss,
    "test_accuracy": test_acc,
    "test_precision": test_precision,
    "test_recall": test_recall,
    "test_f1": test_f1,
})

wandb.finish()
print("\n✓ Training complete! Model saved and logged to W&B.")
